In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
import os
from shapely.geometry import Point
import pyproj
from pyproj import Transformer
import sklearn
from sklearn.cluster import DBSCAN
import numpy as np


In [4]:
df = pd.read_csv("/Users/cherrychoy/Library/CloudStorage/OneDrive-TheUniversityofSydney(Students)/Honours Thesis/crash stats/Walking_count_surveys.csv") 

df.describe()



,OBJECTID,SiteID,Year,TotalCount,Time_0600,Time_0700,Time_0800,Time_0900,Time_1000,Time_1100,...,Time_1400,Time_1500,Time_1600,Time_1700,Time_1800,Time_1900,Time_2000,Time_2100,Time_2200,Time_2300
count,4683.000000,4683.000000,4683.000000,4683.000000,4683.000000,4683.000000,4683.000000,4683.000000,4683.000000,4683.000000,...,4683.000000,4683.000000,4683.000000,4683.000000,4683.000000,4683.000000,4683.000000,4683.000000,4683.000000,4683.000000
mean,9593.586803,62.137732,2018.976938,14110.939569,168.159300,327.396754,619.760837,678.204570,696.366005,843.231689,...,1047.566517,1055.487935,1096.675208,1205.247704,1101.819347,887.401666,712.886184,596.471493,501.127909,397.417895
std,1831.847731,38.118450,3.491049,16989.704564,268.673596,485.643266,930.219993,853.937952,781.700637,1013.964116,...,1388.966598,1384.790445,1498.371195,1656.657702,1476.215762,1206.284538,1003.451595,857.548077,736.833624,645.689896
min,6760.000000,2.000000,2013.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7961.500000,31.000000,2016.000000,3288.000000,36.000000,78.000000,144.000000,180.000000,186.000000,198.000000,...,192.000000,198.000000,204.000000,228.000000,228.000000,198.000000,144.000000,108.000000,84.000000,48.000000
50%,9615.000000,61.000000,2020.000000,7302.000000,90.000000,168.000000,306.000000,366.000000,408.000000,450.000000,...,480.000000,492.000000,480.000000,522.000000,528.000000,444.000000,354.000000,282.000000,222.000000,150.000000
75%,11493.500000,91.000000,2022.000000,19200.000000,198.000000,360.000000,672.000000,834.000000,930.000000,1110.000000,...,1371.000000,1395.000000,1437.000000,1536.000000,1416.000000,1122.000000,900.000000,750.000000,612.000000,456.000000
max,12664.000000,239.000000,2024.000000,121704.000000,6762.000000,7296.000000,13212.000000,9030.000000,7434.000000,10104.000000,...,14166.000000,11136.000000,12000.000000,16806.000000,16092.000000,13776.000000,12492.000000,9660.000000,7734.000000,6552.000000


In [5]:
df = pd.read_csv("/Users/cherrychoy/Library/CloudStorage/OneDrive-TheUniversityofSydney(Students)/Honours Thesis/crash stats/walkcount_weekday_data.csv") 

df.describe()

with pd.option_context('display.max_columns', None):
    print(df.describe())

           OBJECTID       SiteID         Year     TotalCount    Time_0600  \
count   2341.000000  2341.000000  2341.000000    2341.000000  2341.000000   
mean    9592.355831    62.127723  2018.976079   15203.885946   232.040154   
std     1832.572393    38.120439     3.491299   18324.454836   341.963441   
min     6760.000000     2.000000  2013.000000       0.000000     0.000000   
25%     7961.000000    31.000000  2016.000000    3444.000000    54.000000   
50%     9614.000000    61.000000  2020.000000    7056.000000   126.000000   
75%    11492.000000    91.000000  2022.000000   20928.000000   282.000000   
max    12663.000000   239.000000  2024.000000  121704.000000  6762.000000   

         Time_0700     Time_0800    Time_0900    Time_1000     Time_1100  \
count  2341.000000   2341.000000  2341.000000  2341.000000   2341.000000   
mean    475.920974    928.910722   874.109355   729.279795    840.070056   
std     621.934907   1196.034627  1075.151083   863.454867   1085.021211   
mi

In [6]:
#unique site names and locations
# Preview your columns
print(df.columns)

# Get unique combinations of Site ID and Street Name
unique_sites = df[['SiteID', 'Location', 'Description']].drop_duplicates()

# Print them
print(unique_sites)

Index(['OBJECTID', 'SiteID', 'Location', 'Description', 'Period', 'Year',
       'Month', 'TotalCount', 'DateFilter', 'Time_0600', 'Time_0700',
       'Time_0800', 'Time_0900', 'Time_1000', 'Time_1100', 'Time_1200',
       'Time_1300', 'Time_1400', 'Time_1500', 'Time_1600', 'Time_1700',
       'Time_1800', 'Time_1900', 'Time_2000', 'Time_2100', 'Time_2200',
       'Time_2300', 'Average_Daytime_Count', 'Total_Daytime_Count'],
      dtype='object')
      SiteID        Location                                   Description
0          2     Botany Road      Between Bourke Street and Hansard Street
10         3     King Street   Between Whitehorse Street and Newman Street
19        58       City Road            Between Broadway and Myrtle Street
21         4  William Street        Between Crown Street and Palmer Street
31         5       City Road     Between Carillon Avenue and Forbes Street
...      ...             ...                                           ...
1975     215   Hunter St

In [7]:
# List of daytime hourly columns (adjust to match your actual column names)
daytime_columns = ['Time_0600', 'Time_0700', 'Time_0800', 'Time_0900', 'Time_1000', 'Time_1100',
 'Time_1200', 'Time_1300', 'Time_1400', 'Time_1500', 'Time_1600', 'Time_1700',
 'Time_1800', 'Time_1900', 'Time_2000', 'Time_2100', 'Time_2200', 'Time_2300']

# Calculate row-wise average across these columns
df['Average_Daytime_Count'] = df[daytime_columns].mean(axis=1)

# Calculate row-wise sum across columns
df['Total_Daytime_Count'] = df[daytime_columns].sum(axis=1)

df.to_csv("/Users/cherrychoy/Library/CloudStorage/OneDrive-TheUniversityofSydney(Students)/Honours Thesis/crash stats/walkcountsum_avg.csv", index=False)



In [8]:
# Ensure 'Period' column exists and is lowercase/consistent
df['Period'] = df['Period'].str.lower()

# Filter weekday and weekend
weekday_df = df[df['Period'] == 'weekday']
weekend_df = df[df['Period'] == 'weekend']

# Save to CSV
weekday_df.to_csv('/Users/cherrychoy/Library/CloudStorage/OneDrive-TheUniversityofSydney(Students)/Honours Thesis/crash stats/walkcount_weekday_data.csv', index=False)
weekend_df.to_csv('/Users/cherrychoy/Library/CloudStorage/OneDrive-TheUniversityofSydney(Students)/Honours Thesis/crash stats/walkcount_weekend_data.csv', index=False)

print("Files saved")

Files saved


In [9]:
# Load the CSV file
weekday_df = pd.read_csv('/Users/cherrychoy/Library/CloudStorage/OneDrive-TheUniversityofSydney(Students)/Honours Thesis/crash stats/walkcount_weekday_data.csv')

# Sort by Average_Daytime_Count in descending order
ranked_df = weekday_df.sort_values(by='Average_Daytime_Count', ascending=False)

# View top 10 rows
print(ranked_df[['SiteID', 'Location', 'Average_Daytime_Count']].head(10))

ranked_df.to_csv('/Users/cherrychoy/Library/CloudStorage/OneDrive-TheUniversityofSydney(Students)/Honours Thesis/crash stats/walkcount_weekday_ranked.csv', index=False)


      SiteID          Location  Average_Daytime_Count
393       42     George Street            6761.333333
988      104     George Street            5825.666667
1140     105     George Street            5794.333333
2074     105     George Street            5737.666667
401       42     George Street            5669.666667
987      104     George Street            5623.666667
396       42     George Street            5566.666667
413       44     George Street            5504.333333
386       41     George Street            5499.333333
451       47  Pitt Street Mall            5303.333333


In [10]:
# Drop duplicates based on Location and Description
weekday_df1 = weekday_df.drop_duplicates(subset=['Location', 'Description'])

# Sort by average count (descending order for highest footfall)
ranked_df1 = weekday_df1.sort_values(by='Average_Daytime_Count', ascending=False)

print(ranked_df1[['SiteID', 'Location', 'Average_Daytime_Count', 'Total_Daytime_Count']].head(40))


      SiteID          Location  Average_Daytime_Count  Total_Daytime_Count
393       42     George Street            6761.333333               121704
413       44     George Street            5504.333333                99078
991      105     George Street            3968.000000                71424
403       43     George Street            3957.666667                71238
343       37     Market Street            3500.000000                63000
466       50       Park Street            3036.666667                54660
1008     107      Martin Place            2903.666667                52266
363       39       Park Street            2854.666667                51384
383       41     George Street            2699.666667                48594
476       51       Park Street            2661.666667                47910
294       32     George Street            2660.333333                47886
982      104     George Street            2652.000000                47736
1000     106      Martin 

In [11]:
import geopandas as gpd

gdf = gpd.read_file("/Users/cherrychoy/Library/CloudStorage/OneDrive-TheUniversityofSydney(Students)/Honours Thesis/crash stats/Walking_count_surveys.geojson")
print(gdf.columns)
print(gdf.head())

Index(['OBJECTID', 'SiteID', 'Location', 'Description', 'Period', 'Year',
       'Month', 'TotalCount', 'DateFilter', 'Time_0600', 'Time_0700',
       'Time_0800', 'Time_0900', 'Time_1000', 'Time_1100', 'Time_1200',
       'Time_1300', 'Time_1400', 'Time_1500', 'Time_1600', 'Time_1700',
       'Time_1800', 'Time_1900', 'Time_2000', 'Time_2100', 'Time_2200',
       'Time_2300', 'geometry'],
      dtype='object')
   OBJECTID  SiteID     Location                               Description  \
0      6760       2  Botany Road  Between Bourke Street and Hansard Street   
1      6761       2  Botany Road  Between Bourke Street and Hansard Street   
2      6762       2  Botany Road  Between Bourke Street and Hansard Street   
3      6763       2  Botany Road  Between Bourke Street and Hansard Street   
4      6764       2  Botany Road  Between Bourke Street and Hansard Street   

    Period  Year   Month  TotalCount                DateFilter  Time_0600  \
0  weekday  2013  Spring        3504  2